In [4]:
import pandas as pd
df = pd.DataFrame(
        {
            "Name": [
                "Braund, Mr. Owen Harris",
                "Allen, Mr. William Henry",
               "Bonnell, Miss. Elizabeth",
           ],
           "Age": [22, 35, 58],
           "Sex": ["male", "male", "female"],
        }
    )
df["Age"]

0    22
1    35
2    58
Name: Age, dtype: int64

In [6]:
ages = pd.Series([22, 35, 58], name="Age")
ages

0    22
1    35
2    58
Name: Age, dtype: int64

In [7]:
df["Age"].max()

58

In [8]:
ages.max()

58

In [9]:
df.describe()

,Age
count,3.000000
mean,38.333333
std,18.230012
min,22.000000
25%,28.500000
50%,35.000000
75%,46.500000
max,58.000000


In [12]:
import pandas as pd

titanic = pd.read_csv("data/titanic.csv")
titanic.head(8)
titanic.dtypes
titanic.to_excel("titanic.xlsx", sheet_name="passengers", index=False)
# Optional: Anzeigen einer Bestätigungsmeldung
print("DataFrame erfolgreich in Excel-Datei gespeichert.")

ModuleNotFoundError: No module named 'openpyxl'